In [1]:
import plotly.express as px
df = px.data.stocks()

In [2]:
df.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


In [3]:
import sys
# sys.path.append(r'C:\Users\Weronika\Desktop\Intrinsic_Value\Vanguard')

from scrap import *

In [7]:
dropdown_categories = [val for val in list(get_fund_list().keys())]

In [8]:
dropdown_value = dropdown_categories[:3]

In [9]:
pd.merge()

TypeError: merge() missing 2 required positional arguments: 'left' and 'right'

In [15]:
data = pd.DataFrame(columns=['Date'])

for val in dropdown_value:
    print(val)

    df = fund_download(val)
    df.columns = ['Date', val]
    data = pd.merge(data, df, how='outer', on='Date')
#     x = df[df.columns[0]].values
#     y = df[df.columns[1]].values

#     xs.append(x)
#     ys.append(y)

# 

LifeStrategy® 20% Equity Fund
Plotting dataframe...
LifeStrategy® 40% Equity Fund
Plotting dataframe...
LifeStrategy® 60% Equity Fund
Plotting dataframe...


In [16]:
data

,Date,LifeStrategy® 20% Equity Fund,LifeStrategy® 40% Equity Fund,LifeStrategy® 60% Equity Fund
0,2022-08-24,157.2690,185.8731,219.1572
1,2022-08-23,157.9940,186.3882,219.3261
2,2022-08-22,158.9059,187.5881,220.8666
3,2022-08-19,159.9750,189.1456,223.0920
4,2022-08-18,160.9682,189.9699,223.6357
...,...,...,...,...
2820,2011-06-29,99.7151,100.3923,101.1090
2821,2011-06-28,99.6571,100.0215,100.4153
2822,2011-06-27,100.0090,100.0453,100.1093
2823,2011-06-24,100.1144,100.0879,100.0776


In [20]:
fig = px.line(data, x='Date', y = data.columns)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()